In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data = pd.read_json("data/dev.jsonl/dev-stats.jsonl", lines=True)

In [3]:
data.head()

,url,archive,title,date,text,summary,compression,coverage,density,compression_bin,coverage_bin,density_bin
0,http://www.foxsports.com/baseball/xchange/team...,http://web.archive.org/web/19980117162148id_/h...,Pro Sports Xchange notes,1970-08-20 06:01:57.162148,So sayeth Padre general manager Kevin Towers.\...,SAN DIEGO PADRES team notebook,209.000000,0.800000,1.200000,high,medium,abstractive
1,http://www.nytimes.com/2006/06/05/technology/0...,http://web.archive.org/web/20060620021852id_/h...,India Becoming a Crucial Cog in the Machine at...,1970-08-21 04:23:40.021852,"BANGALORE, India, June 4  The world's biggest...",India provides I.B.M. with its fastest-growing...,56.045455,0.954545,16.500000,high,high,extractive
2,http://www.nydailynews.com/archives/news/1995/...,http://web.archive.org/web/20110210093603id_/h...,NEW YORKERS' ONLY REGRET WAS STAYING HOME,1970-08-21 18:10:10.093603,"This story was reported by: NICK CHARLES, AUST...",As many black men marched on Washington yester...,6.152941,0.976471,24.600000,low,high,extractive
3,http://mashable.com/2010/10/16/twitter-top-top...,http://web.archive.org/web/20120123100903id_/h...,Top 10 Twitter Trends This Week [CHART],1970-08-21 20:55:23.100903,Remember when everyone on Earth was glued to T...,Check out the chart to find out what the world...,14.631579,0.842105,1.368421,low,medium,abstractive
4,http://www.reuters.com/article/2011/01/31/us-i...,http://web.archive.org/web/20120321005702id_/h...,Freed American hiker summoned back by Iran court,1970-08-21 20:58:41.005702,"TEHRAN | Mon Jan 31, 2011 9:17am EST\n\nTEHRAN...",TEHRAN (Reuters) - An American woman who was f...,7.902439,1.000000,39.048780,low,high,extractive


In [4]:
data.shape

(108837, 12)

In [5]:
data['text'][0]

'So sayeth Padre general manager Kevin Towers.\n\nLess than two weeks after unofficially setting his starting rotation by trading for No. 1 Kevin Brown and re-signing No. 5 Pete Smith, Towers stirred the pot Jan. 7 by signing Mark Langston to a minor league contract.\n\nBut the Padres have no intention of having Langston pitch at Triple-A Las Vegas. And neither does the 37-year-old left-hander.\n\n"We didn\'t get him to pitch at Las Vegas," said GM Kevin Towers emphatically. "If Mark is back to full health, he\'s going to go for one of the spots in the starting rotation."\n\nThere\'s the rub. Langston had elbow and knee problems last year and made only one start (Aug. 20) after undergoing arthroscopic surgery in May to have bone spurs removed from his left elbow.\n\nIt was his second round of elbow surgery in four seasons.\n\nHowever, the Padres hit the jackpot once before with a over-the-hill left-hander. Fernando Valenzuela was 13-8 with a 3.62 ERA in \'96 and helped pitch the Padres

In [6]:
from nltk.tokenize import sent_tokenize
def split_sentences(text):
    sentences = []
    for s in text:
        sentences.append(sent_tokenize(s))
    sentences = [y for x in sentences for y in x]
    sentences = pd.Series(sentences)
    return sentences

In [7]:
data = data.dropna()

In [8]:
data['sentences'] = split_sentences(data['text'])

In [9]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [10]:
#!unzip glove*.zip

In [11]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [12]:
len(word_embeddings)

400000

In [13]:
data['clean_sentences'] = data['sentences'].str.replace("[^a-zA-z]", " ")
data['clean_sentences'] = data['clean_sentences'].str.lower()

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [16]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [17]:
data['clean_sentences'] = remove_stopwords(data['clean_sentences'])

In [18]:
def sentence_vectors(text):
    sentence_vectors = []
    for i in text:
        if(len(i) != 0):
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v)
    sentence_vectors = pd.Series(sentence_vectors)
    return sentence_vectors

In [ ]:
data['sentence_vectors'] = sentence_vectors(data['clean_sentences'])

In [ ]:
data.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def cosine_similarity(sentences, sentence_vectors):
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    return sim_mat

In [ ]:
data['similarity_matrix'] = cosine_similarity(data['sentences'], data['sentence_vectors'])

In [ ]:
import networkx as nx

data['nx_graph'] = nx.from_numpy_array(data['similarity_mtrix'])

In [ ]:
data['scores'] = nx.pagerank(data['nx_graph'])

In [ ]:
data.head()